
<h1 style="background: linear-gradient(to right, #49A, #0FB); color: white; padding: 20px;">Program 6 Implementation: Bayesian network</h1>

<ol start="6">
    <li>Write a program to construct a Bayesian network considering medical data. Use 
this model to demonstrate the diagnosis of heart patients using standard Heart 
Disease Data Set. You can use Python ML library classes/API.    </li>
</ol>



In [6]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Load and preprocess the data
def preprocess_data(data):
    data['age_bin'] = pd.qcut(data['age'], q=3, labels=['young', 'middle', 'elderly'])
    data['trestbps_bin'] = pd.qcut(data['trestbps'], q=3, labels=['low', 'normal', 'high'])
    data['chol_bin'] = pd.qcut(data['chol'], q=3, labels=['low', 'normal', 'high'])
    processed_data = pd.DataFrame({
        'age': data['age_bin'],
        'sex': data['sex'].map({0: 'F', 1: 'M'}),
        'cp': data['cp'].astype(str),
        'trestbps': data['trestbps_bin'],
        'chol': data['chol_bin'],
        'target': data['target'].astype(str)
    })
    return processed_data

# Create Bayesian Network
def create_bayesian_network():
    model = BayesianNetwork([('age', 'chol'), ('age', 'trestbps'), ('sex', 'chol'),
                            ('sex', 'cp'), ('chol', 'target'), ('trestbps', 'target'),
                            ('cp', 'target')])
    return model

# Train Bayesian Network
def train_bayesian_network(model, data):
    mle = MaximumLikelihoodEstimator(model=model, data=data)
    cpds = [mle.estimate_cpd(node) for node in model.nodes()]
    model.add_cpds(*cpds)
    return model.check_model()

# Make Inference
def make_inference(model, evidence):
    inference = VariableElimination(model)
    result = inference.query(variables=['target'], evidence=evidence)
    return result

# Main function
def main():
    data = pd.read_csv('/content/heart (1).csv')
    processed_data = preprocess_data(data)
    model = create_bayesian_network()
    is_valid = train_bayesian_network(model, processed_data)

    if is_valid:
        print("Bayesian Network successfully created and validated")

        evidence = {'age': 'elderly', 'sex': 'M', 'cp': '3', 'trestbps': 'high', 'chol': 'high'}
        result = make_inference(model, evidence)
        print("\nProbability Distribution for Heart Disease given evidence:")
        print(result)

        correct_predictions, total_predictions = 0, len(processed_data)

        for _, row in processed_data.iterrows():
            evidence = {'age': row['age'], 'sex': row['sex'], 'cp': row['cp'],
                        'trestbps': row['trestbps'], 'chol': row['chol']}
            result = make_inference(model, evidence)
            predicted_class = '1' if result.values[1] > 0.5 else '0'
            correct_predictions += int(predicted_class == row['target'])

        accuracy = correct_predictions / total_predictions
        print(f"\nModel Accuracy: {accuracy:.2f}")
    else:
        print("Error: Invalid Bayesian Network")

if __name__ == "__main__":
    main()

Bayesian Network successfully created and validated

Probability Distribution for Heart Disease given evidence:
+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.4000 |
+-----------+---------------+
| target(1) |        0.6000 |
+-----------+---------------+

Model Accuracy: 0.78
